In [1]:
#--------- Python Libraries ---------#

import requests
import json
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import csv
import pandas as pd
from datetime import date
import datetime
import dateutil.parser
from translation import google, ConnectError

import SpotifyLookup

import string



In [2]:
#--------- Web crawling for URLs ---------#

source = requests.get('https://ekstrabladet.dk/musik/koncert_anmeldelser/').text
soup = BeautifulSoup(source, 'lxml')
print(source)

<!DOCTYPE html>
<html lang="da">
<head>
    <meta charset="utf-8" />
    <title>Koncert-anm. – Ekstra Bladet</title>

<meta name="robots" content="noarchive"/>
    <meta name="robots" content="index, follow"/>
<meta name="description" content="Nyheder, interviews og anmeldelser fra den danske og internationale musikbranche." />

<meta property="fb:app_id" content="123629220366" />
<meta property="fb:pages" content="148503488544345">
<meta property="fb:pages" content="1588645341376097">
<meta property="fb:pages" content="61055003519">
<meta property="fb:pages" content="193743090960312">


<meta property="og:type" content="article" />
    <meta property="og:title" content="Koncert-anm." />
    <meta property="og:description" content="Nyheder, interviews og anmeldelser fra den danske og internationale musikbranche." />
    <meta property="og:url" content="https://ekstrabladet.dk/musik/koncert_anmeldelser/" />
    <meta property="og:image" content="https://ebimg.dk/ux/data/social/eblogo_10

In [3]:
list_of_URLs = []

# Loop through all review links to find concert reviews specifically
link_div = soup.find('div',{'class' : 'flex flex-wrap--wrap flex-justify--between'})

for link in link_div.find_all('a'):
    URL = link.get('href')
    URL = 'https://ekstrabladet.dk' + URL
    list_of_URLs.append(URL)

# Remove duplicate URLs in list
list_of_URLs = list(set(list_of_URLs))
print(list_of_URLs)

['https://ekstrabladet.dk/musik/koncert_anmeldelser/niels-hausgaard-kvajer-sig-i-tivoli/8271226', 'https://ekstrabladet.dk/musik/koncert_anmeldelser/bizart-popfadaese-i-lufthavnen/8130355', 'https://ekstrabladet.dk/musik/koncert_anmeldelser/glem-corona-gaev-jyde-smitter/8267653', 'https://ekstrabladet.dk/musik/koncert_anmeldelser/hold-nu-kaeft-og-snak/8291235', 'https://ekstrabladet.dk/musik/koncert_anmeldelser/tak-rock-lolland-erstatter-parken/8285592']


In [6]:
# 2 test URLs
#list_of_URLs = ['https://soundvenue.com/musik/2020/08/poptalentet-dopha-overvandt-den-akavede-coronastemning-til-lillefredag-i-tivoli-415244']

# Create empty dictionary
critic_review_dict = {} 
critic_review_dict['concerts'] = []

# Loop - For every link in the URL_links do the following with the reviews:
for i in list_of_URLs:

    # Reset values
    critic_name = 'Ekstra Bladet' 
    author = ''
    url = ''
    headline = ''
    text = ''
    rating = 0
    max_rating = 6
    tribute_rating = 0
    artist_name = ''
    venue_name = ''
    venue_place = ''
    publication_date = ''
    concert_date = ''
    lang = 'da'
    genre = []
    
    url = i
    
    # Get the source code of the webpage as text
    source = requests.get(i).text
    soup = BeautifulSoup(source, 'lxml')
    
    ######### Clean Soup for correct text mining #########
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    
    for div in soup.find_all("div", {'class':'oo-line'}): 
        div.decompose()
        
    
    # Find headline
    try:
        headline = soup.h1.text
        headline = headline.lstrip()
        print('Headline found')
    
    except:
        print('No headline found')
    
    # Find text
    try:
        text = soup.find('div',{'class' : 'article-bodytext'})

        text_list = []

        for el in text.find_all('p'):
            text_list.append(el.get_text())

        text = text_list

        for sub_text in text:
            if 'Se også:' in sub_text:
                text.remove(sub_text)

        text = '\n\n'.join(text_list)
        print('Text found')

    except:
        print('No text found')
    
    # Seperate certain string to find, artist, venue, date
    try:
        info_string = soup.find('span',{'class' : 'article-widget-header--text'})
        info_string = info_string.text
        info_string = info_string.split(',')
        print('info string found')
    
    except:
        info_string = ''
        print('No info string found')
        continue
    
    # Find artist name
    try:
        artist_name = info_string[0]
        artist_name = artist_name.lstrip()
        print('artist name found')

    except:
        artist_name = ''
        print('No artist name found')
        continue
    
    # Find venue name
    try:
        venue_name = info_string[1]
        venue_name = venue_name.lstrip()
        print('venue name found')

    except:
        print('No venue name found')
        continue
    
    # Find venue place
    try:
        venue_place = info_string[2]
        venue_place = venue_place.lstrip()
        print('venue place found')

    except:
        print('No venue place found')
        
    
    # Find concert date
    
    concert_month = ''
    
    month_dict = {
        'januar':'jan',
        'februar':'feb',
        'marts':'mar',
        'april':'apr',
        'maj':'may',
        'juni':'jun',
        'juli':'jul',
        'august':'aug',
        'september':'sep',
        'oktober':'oct',
        'november':'nov',
        'december':'dec'
                 }
    
    
    try:
        
        concert_date = info_string[3]
        concert_date = concert_date.lstrip()
        concert_date_words = concert_date.split()
        concert_month = concert_date_words[-1]
        concert_month = month_dict.get(concert_month)
        concert_date = concert_date_words[-2]+concert_month
        print(concert_date)

        # Format date
        concert_date = concert_date # +default_time
        concert_date = dateutil.parser.parse(concert_date, dayfirst=True)
        print('Concert date found')

    except:
        print('No concert date found')
        continue
    
    # Find publication date
    try:    
        publication_date = soup.find('meta', {'property' : 'og:article:published_time'})
        publication_date = publication_date["content"]
        publication_date = dateutil.parser.parse(publication_date)
        print('publication date found')

    except:
        publication_date = ''
        print('No publication_date found') 
        continue
    
    # Find Author
    try:
        author = soup.find('span',{'itemprop' : 'author'})
        author = author.find('span',{'itemprop' : 'name'}).text
        author = author.lstrip()
        print('Author found')

    except:
        print('No author found')
    
    # Find rating
    try:
        rating = soup.find('div',{'class':'rating-stars p-absolute fs-quote'})
        rating = len(rating.find_all('i', {'class' : 'fas fa-star'}))
        tribute_rating = (5/max_rating)*rating
        tribute_rating = round(tribute_rating, 2)
        print('Rating found')

    except:
        print('No rating found')
    
    # Spotify Lookup with artist_name
    spotify_results = SpotifyLookup.spotify_lookup(artist_name)
    artist_name_spotify = spotify_results[0]
    artist_img_spotify = spotify_results[1]
    artist_spotify_id = spotify_results[2]
    subgenres_spotify = spotify_results[3]
    artist_popularity_spotify = spotify_results[4]
    
    print('')
    print('Critic:', critic_name)
    print('Author:', author)
    print('URL:', url)
    print('Artist:', artist_name)
    print('Venue:', venue_name)
    print('Venue Place:', venue_place)
    print('Rating:', str(rating))
    print('Max Rating:', str(max_rating))
    print('Tribute Rating:', str(tribute_rating))
    print('Publication Date:', publication_date)
    print('Concert Date:', concert_date)
    print('artist spotify name:', artist_name_spotify)
    print('artist spotify image:', artist_img_spotify)
    print('artist spotify ID:', artist_spotify_id)
    print('artist spotify subgenres:', subgenres_spotify)
    print('artist spotify popularity:', artist_popularity_spotify)
    print('Language of review:', lang)
    print('Headline:', headline)
    print('Text:', text)
    
    # Set dictionary values

    temp_dict = {}
    temp_dict['critic_name'] = critic_name
    temp_dict['author'] = author
    temp_dict['url'] = url
    temp_dict['artist_name'] = artist_name
    temp_dict['artist_name_spotify'] = artist_name_spotify
    temp_dict['artist_img_spotify'] = artist_img_spotify
    temp_dict['artist_spotify_id'] = artist_spotify_id
    temp_dict['artist_popularity_spotify'] = artist_popularity_spotify
    temp_dict['genre'] = genre
    temp_dict['subgenre_spotify'] = subgenres_spotify
    temp_dict['publication_date'] = str(publication_date)
    temp_dict['concert_date'] = str(concert_date)
    temp_dict['venue_name'] = venue_name
    temp_dict['venue_place'] = venue_place
    temp_dict['rating'] = rating
    temp_dict['tribute_rating'] = tribute_rating
    temp_dict['max_rating'] = max_rating
    temp_dict['review_language'] = lang
    temp_dict['headline'] = headline
    temp_dict['text'] = text

    critic_review_dict['concerts'].append(temp_dict)

    
    
    


Headline found
Text found
info string found
artist name found
venue name found
venue place found
1.sep
Concert date found
publication date found
Author found
Rating found
Name preprocessing: Niels Hausgaard = niels hausgaard
found 1 results
searching for exact string match...
Found exact string match on index 0 Niels Hausgaard

Critic: Ekstra Bladet
Author: Thomas Treo
URL: https://ekstrabladet.dk/musik/koncert_anmeldelser/niels-hausgaard-kvajer-sig-i-tivoli/8271226
Artist: Niels Hausgaard
Venue: Glassalen
Venue Place: København
Rating: 3
Max Rating: 6
Tribute Rating: 2.5
Publication Date: 2020-09-01 21:34:11+00:00
Concert Date: 2020-09-01 00:00:00
artist spotify name: Niels Hausgaard
artist spotify image: https://i.scdn.co/image/f9fe6688d2db3f95f53b7544ba6a2c70679b4bf4
artist spotify ID: 4AsdZmU2lWN40Q6yCxcR0u
artist spotify subgenres: ['classic danish pop', 'dansktop']
artist spotify popularity: 27
Language of review: da
Headline: Niels Hausgaard kvajer sig i Tivoli
Text: Som en af k

Headline found
Text found
info string found
artist name found
No venue name found


In [7]:
# Create json object with utf-8
with open('ekstra_bladet_critic_reviews.json', 'w', encoding='utf-8') as json_file:
    json.dump(critic_review_dict, json_file, ensure_ascii=False)

In [8]:
with open('ekstra_bladet_critic_reviews.json') as json_file:
    data = json.load(json_file)
    jstr = json.dumps(data, indent=4)
    print(jstr)


{
    "concerts": [
        {
            "critic_name": "Ekstra Bladet",
            "author": "Thomas Treo",
            "url": "https://ekstrabladet.dk/musik/koncert_anmeldelser/niels-hausgaard-kvajer-sig-i-tivoli/8271226",
            "artist_name": "Niels Hausgaard",
            "artist_name_spotify": "Niels Hausgaard",
            "artist_img_spotify": "https://i.scdn.co/image/f9fe6688d2db3f95f53b7544ba6a2c70679b4bf4",
            "artist_spotify_id": "4AsdZmU2lWN40Q6yCxcR0u",
            "artist_popularity_spotify": 27,
            "genre": [],
            "subgenre_spotify": [
                "classic danish pop",
                "dansktop"
            ],
            "publication_date": "2020-09-01 21:34:11+00:00",
            "concert_date": "2020-09-01 00:00:00",
            "venue_name": "Glassalen",
            "venue_place": "K\u00f8benhavn",
            "rating": 3,
            "tribute_rating": 2.5,
            "max_rating": 6,
            "review_language": "da",
      